# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [10]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

In [306]:
def load_rules(config_file):
    import json
    try:
        with open(config_file) as f:
            config = json.load(f)
    except:
        config = {
    "Organ Failure History" : {
        "Nonoperative": 5,
        "Emergency": 5,
        "Elective": 2,
        "None": 0
    },
    "Age": [
        { "min":  0, "max": 45,  "points": 0 },
        { "min": 45, "max": 55,  "points": 2 },
        { "min": 55, "max": 65,  "points": 3 },
        { "min": 65, "max": 75,  "points": 5 },
        { "min": 75, "max": 999, "points": 6 }
    ],
    "Creatinine": {
        "Acute Renal Failure": [
            { "min": -999, "max": 0.6, "points": 2 },
            { "min":  0.6, "max": 1.5,  "points": 0 },
            { "min":  1.5, "max": 2,  "points": 4 },
            { "min":  2, "max": 3.5,  "points": 6 },
            { "min":  3.5, "max": 999,  "points": 8 } 
        ],
        "Chronic Renal Failure": [
            { "min": -999, "max": 0.6, "points": 2 },
            { "min":  0.6, "max": 1.5,  "points": 0 },
            { "min":  1.5, "max": 2,  "points": 2 },
            { "min":  2, "max": 3.5,  "points": 3 },
            { "min":  3.5, "max": 999,  "points": 4 }
        ],
        "Normal Renal Function": [
            { "min": -999, "max": 0.6, "points": 2 },
            { "min":  0.6, "max": 1.5,  "points": 0 }
        ]
    },
    "Temperature": [
        { "min": -999, "max": 30,  "points": 4 },
        { "min": 30, "max": 32,  "points": 3 },
        { "min": 32, "max": 34,  "points": 2 },
        { "min": 34, "max": 36,  "points": 1 },
        { "min": 36, "max": 38.5, "points": 0 },
        { "min": 38.5, "max": 39,  "points": 1 },
        { "min": 39, "max": 41,  "points": 3 },
        { "min": 41, "max": 999,  "points": 4 }
    ],
    "pH": [
        { "min": -999, "max": 7.15,  "points": 4 },
        { "min": 7.15, "max": 7.25,  "points": 3 },
        { "min": 7.25, "max": 7.33,  "points": 2 },
        { "min": 7.33, "max": 7.5, "points": 0 },
        { "min": 7.5, "max": 7.6,  "points": 1 },
        { "min": 7.6, "max": 7.7,  "points": 3 },
        { "min": 7.7, "max": 999,  "points": 4 }
    ],
    "Heart Rate": [
        { "min": 0, "max": 40,  "points": 4 },
        { "min": 40, "max": 55,  "points": 3 },
        { "min": 55, "max": 70,  "points": 2 },
        { "min": 70, "max": 110, "points": 0 },
        { "min": 110, "max": 140,  "points": 2 },
        { "min": 140, "max": 180,  "points": 3 },
        { "min": 180, "max": 999,  "points": 4 }
    ],
    "Respiratory Rate": [
        { "min": 0, "max": 6,  "points": 4 },
        { "min": 6, "max": 10,  "points": 2 },
        { "min": 10, "max": 12,  "points": 1 },
        { "min": 12, "max": 25, "points": 0 },
        { "min": 25, "max": 35,  "points": 1 },
        { "min": 35, "max": 50,  "points": 3 },
        { "min": 50, "max": 999,  "points": 4 }
    ],
    "Sodium": [
        { "min": 0, "max": 111,  "points": 4 },
        { "min": 111, "max": 120,  "points": 3 },
        { "min": 120, "max": 130,  "points": 2 },
        { "min": 130, "max": 150, "points": 0 },
        { "min": 150, "max": 155,  "points": 1 },
        { "min": 155, "max": 160,  "points": 2 },
        { "min": 160, "max": 180,  "points": 3 },
        { "min": 180, "max": 999,  "points": 4 }
    ],
    "Potassium": [
        { "min": 0, "max": 2.5,  "points": 4 },
        { "min": 2.5, "max": 3,  "points": 2 },
        { "min": 3, "max": 3.5,  "points": 1 },
        { "min": 3.5, "max": 5.5, "points": 0 },
        { "min": 5.5, "max": 6,  "points": 1 },
        { "min": 6, "max": 7,  "points": 3 },
        { "min": 7, "max": 999,  "points": 4 }
    ],
    "Hematocrit": [
        { "min": 0, "max": 20,  "points": 4 },
        { "min": 20, "max": 30,  "points": 2 },
        { "min": 30, "max": 46, "points": 0 },
        { "min": 46, "max": 50,  "points": 1 },
        { "min": 50, "max": 60,  "points": 2 },
        { "min": 60, "max": 999,  "points": 4 }
    ],
    "White Blood Count": [
        { "min": 0, "max": 1,  "points": 4 },
        { "min": 1, "max": 3,  "points": 2 },
        { "min": 3, "max": 15, "points": 0 },
        { "min": 15, "max": 20,  "points": 1 },
        { "min": 20, "max": 40,  "points": 2 },
        { "min": 40, "max": 999,  "points": 4 }
    ],
    "FiO2": {
        "<50" : [
            { "min": 0, "max": 55,  "points": 4 },
            { "min": 55, "max": 60.00001, "points": 3 },
            { "min": 61, "max": 70.00001, "points": 1 },
            { "min": 70.00001, "max": 999,  "points": 0 }
        ],
        "50+" : [
            { "min": 0, "max": 200,  "points": 0 },
            { "min": 200, "max": 349, "points": 2 },
            { "min": 349, "max": 499, "points": 3 },
            { "min": 499, "max": 999,  "points": 4 }
        ],
    }
}
    
    return config

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [36]:
print("Instead of using 'config_file' as a parameter, I used 'rules_or_config' to make it more flexible. This works by either accepting a dictionary of the rules to use, or will accept a string containing a JSON file location to load the rules dictionary from the file. The purpose of this setup is so that when testing and calling functions, you can start with any of them and only need to load the rules once then pass them along, instead of constantly opening the file and reading repeatedly.")

Instead of using 'config_file' as a parameter, I used 'rules_or_config' to make it more flexible. This works by either accepting a dictionary of the rules to use, or will accept a string containing a JSON file location to load the rules dictionary from the file. The purpose of this setup is so that when testing and calling functions, you can start with any of them and only need to load the rules once then pass them along, instead of constantly opening the file and reading repeatedly.


In [305]:
import json
def organ_failure_score(rules_or_config, organ_failure_history):
    """
    organ_failure_score(rules_or_config, organ_failure_history) -> int
    accepts either a string or dictionary containing a set of scoring criteria, and a string with organ history classification, and returns a integer representing the organ history subset of points in the Apache II scoring system.
    rules_or_config [str or dict]: can be a string that points to a JSON file or a dictionary object; dictionary must contain first-level key of "Organ Failure History" under which the second level keys represented by organ_failure_history are contained.
    organ_failure_history [str]: valid values are "Nonoperative", "Emergency", "Elective", or "None"
    """
    if type(rules_or_config) == dict:
        rules = rules_or_config       
    else:
        rules = load_rules(rules_or_config)
    x = organ_failure_history
    score = rules.get("Organ Failure History").get(x)
    return score

def get_ranges_score(measurement, ranges):
    for range in ranges:
        if (float(measurement) >= float(range.get("min")) ) & (float(measurement) < float(range.get("max")) ):
            return int(range.get("points"))
    return 9999

def rule_points(rules_or_config, measurement, criteria, subcriteria = ""):
    """
    rule_points(rules_or_config, measurement, criteria, subcriteria = "") -> int
    accepts either a string or dictionary containing a set of scoring criteria, a number representing a measurement, a string representing a criteria, and an optional subcriteria, and returns a integer representing the coresponding subset of points in the Apache II scoring system, though it can be used for other purposes.
    rules_or_config [str or dict]: can be a string that points to a JSON file or a dictionary object; dictionary must contain first-level key identical to the criteria parameter being passed
    measurement [int or float]: a numerical value which will be compared to a list of dictionaries containing min and max values (min inclusive, max exclusive) and corresponding point values.
    criteria [str]: string must be identical to one of the first-order keys contained in the dictionary called through rules_or_config
    subcriteria [str](optional): for criteria that have another set of identifier keys before the lsit of range dictionaries; must exactly match the second-order keys contained in the dictionary called through rules_or_config
    the criteria key or, if used, subcriteria key, must have a corresponding value of a list containing dictionaries in the following format: [{"min": x, "max": y, "points": z}, ...]
    
    """
    if type(rules_or_config) == dict:
        rules = rules_or_config       
    else:
        rules = load_rules(rules_or_config)
        
    if subcriteria == "":
        ranges = rules.get(criteria)
    else:
        ranges = rules.get(criteria).get(subcriteria)
        
    score = get_ranges_score(measurement, ranges)
    if score == 9999:
        print("Criteria: " + criteria + " " + subcriteria + " Measurement: " + str(measurement) + " Something seems off. Maybe recheck those numbers...")
    
    return score
    



3


### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [319]:
assert(rule_points("apache.json",37,"Temperature") == 0)
assert(rule_points("apache.json",73,"Age") == 5)
assert(rule_points("apache.json","140","Heart Rate") == 3)
assert(rule_points("apache.json",12,"Respiratory Rate") == 0)
assert(rule_points("apache.json",250,"FiO2","50+") == 2)
assert(rule_points("apache.json",65,"FiO2","<50") == 1)
assert(rule_points("apache.json",2,"Creatinine", "Acute Renal Failure") == 6)
assert(rule_points("apache.json",2,"Creatinine", "Chronic Renal Failure") == 3)

#Ok this is getting tedious. I'm already checking it against a bunch of test cases below, and admittedly I'm writing this block after having finished everything else, so please don't knock me for not writing more of these.


---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [307]:
def apache_score(rules_or_config, organ_failure_history, age, temp, ph, hr, rr, na, k, cr, arf, hct, wbc, glasgow, fio2, pao2 = -1, aa = -1):
    """
    apache_score(rules_or_config, organ_failure_history, age, temp, ph, hr, rr, na, k, cr, arf, hct, wbc, glasgow, fio2, pao2 = -1, aa = -1) -> int
    accepts as parameters a dictionary or filepath to a JSON file with scoring critera, and the relevant metrics to calculate an Apache II score, minus the MAP, and returns the Apache II score, assuming normal MAP.
    requires rules_points function and organ_failure_score function
    rules_or_config [str or dict]: can be a string that points to a JSON file or a dictionary object; dictionary must contain first-level keys identical to the criteria parameters listed below.
    organ_failure_history [str]: valid values are "Nonoperative", "Emergency", "Elective", or "None"      key: "Organ Failure History"
    age [int or float]: patient age in years     key: "Age"
    temp [int or float]: Rectal temperature, °C     key: "Temperature"
    ph [int or float]: Arterial pH     key: "pH"
    hr [int or float]: Heart rate, beats per minute     key: "Heart Rate"
    rr [int or float]: Respiratory rate, breaths per minute     key: "Respiratory Rate"
    na [int or float]: Serum sodium, mmol/L     key: "Sodium"
    k [int or float]: Serum potassium, mmol/L     key: "Potassium"
    cr [int or float]: Serum creatinine, mg/100 mL     key: "Creatinine"
    arf [str]: status or renal function or renal failure, valid values are "Acute Renal Failure", "Chronic Renal Failure", or "Normal Renal Function". These are subcriteria keys under "Creatinine"
    hct [int or float]: Hematocrit, %     key: "Hematocrit"
    wbc [int or float]: White blood count, total/cubic mm in 1000's     key: "White Blood Count"
    glasgow [int or float]: Glasgow Coma Scale (GCS); should be value of 1 to 15, does not use dictionary key
    fio2 [int or float]: Oxygenation, specifically fractional inspiration of oxygen (%)     key: "FiO2"
    pao2 [int or float]: arterial partial pressure of oxygen (mmHG), only required if fio2 is under 50%     key is second-order under FiO2 and is "<50"
    aa [int or float]: Alveolar-arterial gradient, only required if fio2 is at least 50%     key is second-order under FiO2 and is "50+"
    
    Warning: this function does not perform robust input-checking, so do not depend on it to screen for incorrect values in the data being fed
    """
    if type(rules_or_config) == dict:
        rules = rules_or_config       
    else:
        rules = load_rules(rules_or_config)
    
    subscores = list(range(13))
    subscores[0] = organ_failure_score(rules, organ_failure_history)
    subscores[1] = rule_points(rules, age, "Age")
    subscores[2] = rule_points(rules, temp, "Temperature")
    subscores[3] = rule_points(rules, ph, "pH")
    subscores[4] = rule_points(rules, hr, "Heart Rate")
    subscores[5] = rule_points(rules, rr, "Respiratory Rate")
    subscores[6] = rule_points(rules, na, "Sodium")
    subscores[7] = rule_points(rules, k, "Potassium")
    subscores[8] = rule_points(rules, cr, "Creatinine", arf)
    subscores[9] = rule_points(rules, hct, "Hematocrit")
    subscores[10] = rule_points(rules, wbc, "White Blood Count")
    subscores[11] = 15 - glasgow
    
    if fio2 < 50:
        subscores[12] = rule_points(rules, pao2, "FiO2", "<50")
    else:
        subscores[12] = rule_points(rules, aa, "FiO2", "50+")
        
    apache2_score = sum(subscores)
    #print(organ_failure_history, age, temp, ph, hr, rr, na, k, cr, arf, hct, wbc, glasgow, fio2, pao2, aa, subscores, apache2_score)
    return apache2_score

    

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [251]:
x = apache_score("apache.json", "Elective", 3, 4, 7.2, 74, 53, 106, 3.2, 1.2, "Acute Renal Failure", 60, 22, 11, 66, 59, 482)

Elective 3 4 7.2 74 53 106 3.2 1.2 Acute Renal Failure 60 22 11 66 59 482 [2, 0, 4, 3, 0, 4, 4, 1, 0, 4, 2, 4, 3] 31


---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [308]:
def pt_csv_apache2(rules_or_config, uri):
    """
    pt_csv_apache2(rules_or_config, uri) -> list[int]
    accepts as parameters a dictionary or filepath to a JSON file with scoring critera, and a URI pointing to a CSV file of patient data and returns a list of their APACHE II scores assuming normal MAP using the apache_score function
    rules_or_config [str or dict]: can be a string that points to a JSON file or a dictionary object; dictionary must contain keys identical to the criteria parameters listed in the apache_score function docstring
    uri [str]: URI pointing to a CSV file containing patient data with relevant metrics to calculate APACHE II score, with the exception of MAP, with column headings matching as listed below:
    Organ Failure History
    Age
    Temperature
    pH
    Heart rate
    Respiratory rate
    Sodium
    Potassium
    Creatinine
    Acute renal failure
    Hematocrit
    White Blood Count
    Glasgow Coma Scale
    FiO2
    PaO2
    A-a gradient
    
    see apache_score doctring for more information
    """
    import pandas as pd
    import json
    if type(rules_or_config) == dict:
        rules = rules_or_config       
    else:
        rules = load_rules(rules_or_config)
        
    data = pd.read_csv(uri)
    df = pd.DataFrame(data)
    #print(df)
    #score_df = pd.DataFrame(columns=["Patient","Apache II Score"])
    score_list = list()
    for patient in df.index:
        #score = apache_score(rules, df.loc[patient]['Organ Failure History'], df.loc[patient]['Age'], df.loc[patient]['Temperature'], df.loc[patient]['pH'], df.loc[patient]['Heart Rate'], df.loc[patient]['Respiratory Rate'], df.loc[patient]['Sodium'], df.loc[patient]['Potassium'], df.loc[patient]['Creatinine'], df.loc[patient]['Acute Renal Failure'], df.loc[patient]['Hematocrit'], df.loc[patient]['White Blood Count'], df.loc[patient]['Glasgow Coma Scale'], df.loc[patient]['FiO2'], df.loc[patient]['PaO2'], df.loc[patient]['A-a Gradient'])
        score = apache_score(rules, str(df['Organ Failure History'][patient]), float(df['Age'][patient]), float(df['Temperature'][patient]), float(df['pH'][patient]), float(df['Heart Rate'][patient]), float(df['Respiratory Rate'][patient]), float(df['Sodium'][patient]), float(df['Potassium'][patient]), float(df['Creatinine'][patient]), str(df['Acute Renal Failure'][patient]), float(df['Hematocrit'][patient]), float(df['White Blood Count'][patient]), float(df['Glasgow Coma Scale'][patient]), float(df['FiO2'][patient]), float(df['PaO2'][patient]), float(df['A-a Gradient'][patient]))
        #print(str(df['Organ Failure History'][patient]), float(df['Age'][patient]), float(df['Temperature'][patient]), float(df['pH'][patient]), float(df['Heart Rate'][patient]), float(df['Respiratory Rate'][patient]), float(df['Sodium'][patient]), float(df['Potassium'][patient]), float(df['Creatinine'][patient]), str(df['Acute Renal Failure'][patient]), float(df['Hematocrit'][patient]), float(df['White Blood Count'][patient]), float(df['Glasgow Coma Scale'][patient]), float(df['FiO2'][patient]), float(df['PaO2'][patient]), float(df['A-a Gradient'][patient]))
        
        score_list.append(int(score))
        #score_df.append([patient, score])

    #print(score_list)
    return(score_list)

In [257]:
###
#def csv_apache2(rules_or_config, uri):
#    """
#    
#    """
#    import pandas as pd
#    import json
#    import csv
#    if type(rules_or_config) == dict:
#        rules = rules_or_config       
#    else:
#        rules = load_rules(rules_or_config)
#        
#    #with open(uri) as f:
#    data = pd.read_csv(uri)
#    df = pd.DataFrame(data)
#    reader = csv.DictReader(df)
#
#    from csv import DictReader
#
#    with open(data, 'r') as read_obj:
#        csv_dict_reader = DictReader(read_obj)
#        for row in csv_dict_reader:
#            print(row["Organ Failure History"])
###


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [262]:
def test_csv_read(rules_or_config, uri):
    """
    
    """
    import pandas as pd
    import json
    if type(rules_or_config) == dict:
        rules = rules_or_config       
    else:
        rules = load_rules(rules_or_config)
        
    data = pd.read_csv(uri)
    df = pd.DataFrame(data)
    #print(df)
    score_df = pd.DataFrame(columns=["Patient","Apache II Score"])
    score_list = list()
    for patient in df.index:
        print(df['Age'][patient])

In [288]:
x = pt_csv_apache2("apache.json","https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")
print(x)

[35, 31, 47, 34, 44, 35, 31, 49, 40, 48, 42, 43, 32, 41, 42, 49, 37, 37, 38, 43, 41, 31, 38, 30, 41, 41, 34, 46, 40, 47, 36, 43, 41, 46, 44, 40, 39, 37, 41, 30, 46, 30, 41, 44, 35, 36, 40, 40, 30, 50, 52, 43, 46, 34, 33, 42, 41, 31, 46, 46, 34, 36, 33, 38, 26, 29, 46, 25, 40, 38, 39, 34, 39, 53, 44, 49, 37, 36, 36, 51, 33, 36, 43, 41, 24, 50, 29, 40, 36, 50, 29, 37, 34, 45, 34, 40, 37, 37, 47, 31, 33, 50, 28, 37, 33, 44, 40, 38, 40, 31, 44, 37, 44, 47, 25, 34, 32, 41, 38, 34, 38, 39, 41, 34, 29, 44, 22, 40, 34, 43, 40, 31, 35, 33, 27, 44, 52, 40, 53, 40, 40, 48, 31, 42, 49, 43, 35, 31, 36, 42, 34, 31, 30, 28, 38, 47, 32, 47, 31, 41, 40, 33, 35, 42, 49, 55, 51, 37, 53, 29, 33, 46, 36, 32, 33, 33, 42, 32, 41, 35, 39, 40, 40, 35, 28, 33, 42, 29, 30, 39, 34, 46, 30, 37, 31, 37, 41, 47, 45, 29, 45, 33, 33, 38, 28, 42, 32, 45, 36, 39, 43, 32, 37, 43, 34, 40, 27, 24, 39, 39, 35, 36, 39, 47, 27, 28, 50, 53, 36, 44, 38, 51, 43, 38, 38, 36, 40, 36, 40, 31, 39, 37, 38, 46, 28, 32, 37, 29, 26, 25,

In [309]:
import csv
import requests
import codecs


scores = pt_csv_apache2("apache.json","https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")

check_answers = list()
check_text = requests.get("https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv").iter_lines()
answers = csv.reader(codecs.iterdecode(check_text, 'utf-8'))

next(answers, None)

for line in answers:
    x = int(float(line[0]))
    check_answers.append(x)

   
#print(check_answers)

L = max(len(scores),len(check_answers))

print("Whole sequence is equal: ",scores==check_answers)
print("\n------------------------------------------------------- \n")
for i in range(L):
    print(("Calculated score: {}  Check score: {}  Match?:  {}".format(scores[i],check_answers[i],(scores[i]==check_answers[i]))))

    




Whole sequence is equal:  True

------------------------------------------------------- 

Calculated score: 35  Check score: 35  Match?:  True
Calculated score: 31  Check score: 31  Match?:  True
Calculated score: 47  Check score: 47  Match?:  True
Calculated score: 34  Check score: 34  Match?:  True
Calculated score: 44  Check score: 44  Match?:  True
Calculated score: 35  Check score: 35  Match?:  True
Calculated score: 31  Check score: 31  Match?:  True
Calculated score: 49  Check score: 49  Match?:  True
Calculated score: 40  Check score: 40  Match?:  True
Calculated score: 48  Check score: 48  Match?:  True
Calculated score: 42  Check score: 42  Match?:  True
Calculated score: 43  Check score: 43  Match?:  True
Calculated score: 32  Check score: 32  Match?:  True
Calculated score: 41  Check score: 41  Match?:  True
Calculated score: 42  Check score: 42  Match?:  True
Calculated score: 49  Check score: 49  Match?:  True
Calculated score: 37  Check score: 37  Match?:  True
Calculate